In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os
import shutil

# 원본 데이터셋 경로
source_path = "/content/drive/MyDrive/DMS/Phone_detection_v1i_yolov8"  # 원본 데이터 경로

# 새로 복사할 데이터셋 경로
destination_path = "/content/drive/MyDrive/DMS/Phone_detection_v1i_yolov8_modified"  # 수정된 데이터 저장 경로

# 디렉토리 구조 생성
os.makedirs(os.path.join(destination_path, "train/images"), exist_ok=True)
os.makedirs(os.path.join(destination_path, "train/labels"), exist_ok=True)
os.makedirs(os.path.join(destination_path, "valid/images"), exist_ok=True)
os.makedirs(os.path.join(destination_path, "valid/labels"), exist_ok=True)
os.makedirs(os.path.join(destination_path, "test/images"), exist_ok=True)
os.makedirs(os.path.join(destination_path, "test/labels"), exist_ok=True)

# 함수: 라벨 파일 수정 (모든 클래스 ID를 0으로 변경)
def modify_label_file(src_label_path, dest_label_path):
    with open(src_label_path, "r") as f:
        lines = f.readlines()

    modified_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) > 0:
            parts[0] = "0"  # 클래스 ID를 0으로 변경
            modified_lines.append(" ".join(parts))

    # 수정된 내용을 새 파일에 저장
    with open(dest_label_path, "w") as f:
        f.write("\n".join(modified_lines))

# 데이터 복사 및 라벨 수정
for split in ["train", "valid", "test"]:  # train, valid, test 디렉토리를 순회
    image_dir = os.path.join(source_path, split, "images")
    label_dir = os.path.join(source_path, split, "labels")

    new_image_dir = os.path.join(destination_path, split, "images")
    new_label_dir = os.path.join(destination_path, split, "labels")

    if not os.path.exists(image_dir) or not os.path.exists(label_dir):
        print(f"경로가 존재하지 않습니다: {image_dir} 또는 {label_dir}")
        continue

    # 이미지 복사
    for image_file in os.listdir(image_dir):
        src_image_path = os.path.join(image_dir, image_file)
        dest_image_path = os.path.join(new_image_dir, image_file)
        shutil.copy(src_image_path, dest_image_path)  # 이미지 파일 복사

    # 라벨 복사 및 수정
    for label_file in os.listdir(label_dir):
        if label_file.endswith(".txt"):
            src_label_path = os.path.join(label_dir, label_file)
            dest_label_path = os.path.join(new_label_dir, label_file)
            modify_label_file(src_label_path, dest_label_path)  # 라벨 수정 후 저장

print(f"데이터셋이 {destination_path}에 성공적으로 복사되고 라벨이 수정되었습니다.")

데이터셋이 /content/drive/MyDrive/DMS/Phone_detection_v1i_yolov8_modified에 성공적으로 복사되고 라벨이 수정되었습니다.


In [ ]:
# data.yaml 파일 복사
src_yaml_path = os.path.join(source_path, "data.yaml")
dest_yaml_path = os.path.join(destination_path, "data.yaml")

if os.path.exists(src_yaml_path):
    shutil.copy(src_yaml_path, dest_yaml_path)
    print(f"data.yaml 파일 복사 완료: {dest_yaml_path}")
else:
    print("data.yaml 파일이 존재하지 않습니다.")

data.yaml 파일 복사 완료: /content/drive/MyDrive/DMS/Phone_detection_v1i_yolov8_modified/data.yaml


In [ ]:
import os

# 각 세트의 이미지 디렉토리 경로
splits = ["train", "valid", "test"]

# 이미지 수 계산 함수
def count_images_in_split(dataset_path, splits):
    image_counts = {}
    for split in splits:
        image_dir = os.path.join(dataset_path, split, "images")
        if os.path.exists(image_dir):
            # 이미지 파일 개수 계산
            image_count = len([file for file in os.listdir(image_dir) if file.endswith((".jpg", ".png"))])
            image_counts[split] = image_count
        else:
            print(f"{split} 디렉토리가 존재하지 않습니다: {image_dir}")
            image_counts[split] = 0
    return image_counts

# 이미지 수 계산 및 출력
print("origin 이미지 개수")
image_counts = count_images_in_split(source_path, splits)
for split, count in image_counts.items():
    print(f"{split.capitalize()} 세트의 이미지 개수: {count}")

print("\nmodified 이미지 개수")
image_counts = count_images_in_split(destination_path, splits)
for split, count in image_counts.items():
    print(f"{split.capitalize()} 세트의 이미지 개수: {count}")

origin 이미지 개수
Train 세트의 이미지 개수: 3911
Valid 세트의 이미지 개수: 386
Test 세트의 이미지 개수: 200

modified 이미지 개수
Train 세트의 이미지 개수: 3911
Valid 세트의 이미지 개수: 386
Test 세트의 이미지 개수: 200


In [ ]:
# 2. 데이터셋 경로 설정 (Colab 환경 기준, 사용자가 업로드한 데이터셋 위치 확인 필요)
dataset_path = destination_path  # 압축 해제된 데이터 경로를 여기에 설정하세요
data_yaml_path = os.path.join(dataset_path, "data.yaml")  # data.yaml 파일이 존재해야 함

# 3. YOLOv8 모델 로드 및 학습 설정
model = YOLO("yolov8m.pt")  # Pretrained YOLOv8 모델 사용

# 4. 모델 학습 (파인튜닝)
model.train(
    data=data_yaml_path,  # 데이터셋 경로
    epochs=40,            # 학습 횟수
    batch=32,             # 배치 크기
    imgsz=224,            # 추론 환경과 동일한 크기로 설정
    single_cls=True,
    project='/content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad',
    device="cuda" if torch.cuda.is_available() else "cpu",  # GPU 사용 가능 여부 확인
    workers=8             # 데이터 로딩 워커 수
)

print("모델 학습 및 저장 완료! best.pt 파일을 확인하세요.")
# model.train만 해도 run에 best.pt로 저장


Ultralytics 8.3.89 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/DMS/Phone_detection_v1i_yolov8_modified/data.yaml, epochs=40, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=/content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning /content/drive/.shortcut-targets-by-id/1cgNOYE-YX-MwUOcFE_5QCHeQKByw-spz/DMS/Phone_detection_v1i_yolov8_modified/train/labels... 3911 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3911/3911 [00:55<00:00, 70.29it/s] 


train: New cache created: /content/drive/.shortcut-targets-by-id/1cgNOYE-YX-MwUOcFE_5QCHeQKByw-spz/DMS/Phone_detection_v1i_yolov8_modified/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/.shortcut-targets-by-id/1cgNOYE-YX-MwUOcFE_5QCHeQKByw-spz/DMS/Phone_detection_v1i_yolov8_modified/valid/labels... 386 images, 0 backgrounds, 0 corrupt: 100%|██████████| 386/386 [00:04<00:00, 89.05it/s] 


val: New cache created: /content/drive/.shortcut-targets-by-id/1cgNOYE-YX-MwUOcFE_5QCHeQKByw-spz/DMS/Phone_detection_v1i_yolov8_modified/valid/labels.cache
Plotting labels to /content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad/train
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      40/40      2.91G     0.8013     0.3565     0.9533          8        224: 100%|██████████| 123/123 [00:20<00:00,  6.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.00it/s]

                   all        386        395      0.971      0.946       0.97      0.582



40 epochs completed in 0.273 hours.
Optimizer stripped from /content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad/train/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad/train/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad/train/weights/best.pt...
Ultralytics 8.3.89 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.03it/s]


                   all        386        395      0.976      0.944      0.974      0.582
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/drive/MyDrive/DMS/YOLOv8_fine/phone-detection-umad/train
모델 학습 및 저장 완료! best.pt 파일을 확인하세요.
